## Stream attributes from Web events using Signals

This notebook creates a new view using the SDK, tests it on the atomic events table and applies.

### Flow of data

```mermaid
flowchart LR
    sp(Snowplow Pipeline)
    stream[/Stream processing/]
    signals(Signals)

    sp --> stream
    stream --> signals
```

---

# Installation and setup

In [ ]:
%pip install snowplow-signals

In [ ]:
from snowplow_signals import Signals
import os

try:
    from google.colab import userdata
    sp_signals = Signals(
            api_url=userdata.get('SP_API_URL'),
            api_key=userdata.get('SP_API_KEY'),
            api_key_id=userdata.get('SP_API_KEY_ID'),
            org_id=userdata.get('SP_ORG_ID'),
        )
except ImportError:
    from dotenv import load_dotenv
    load_dotenv()
    sp_signals = Signals(
        api_url=os.environ['SP_API_URL'],
        api_key=os.environ['SP_API_KEY'],
        api_key_id=os.environ['SP_API_KEY_ID'],
        org_id=os.environ['SP_ORG_ID'],
)

### Define a new attribute

This block creates a single attribute definition including the logic how it should be calculated (it's filters and aggregation).

The attribute calculates the number of add to cart ecommerce events.

In [ ]:
from snowplow_signals import (
    Attribute,
    Criteria,
    Criterion,
    Event,
)

clicked_links_on_homepage = Attribute(
    name="clicked_links",
    type="string_list",
    events=[
        Event(
            vendor="com.snowplowanalytics.snowplow",
            name="link_click",
            version="1-0-1",
        )
    ],
    aggregation="unique_list",
    property="unstruct_event_com_snowplowanalytics_snowplow_link_click_1:targetUrl",
    criteria=Criteria(
        all=[
            Criterion(
                property="page_urlpath",
                operator="=",
                value="/",
            ),
        ],
    ),
)

### Wrapping the attribute in a view

All attributes need to be included in views that can be considered as "tables" of attributes.

Views are immutable and versioned.

In [ ]:
from snowplow_signals import View, session_entity

view = View(
    name="my_web_attributes",
    version=1,
    entity=session_entity,
    attributes=[
        clicked_links_on_homepage,
    ],
)

### Testing the view

Execute the view on the last one hour of data from the atomic events table to verify that it works correctly.

In [ ]:
data = sp_signals.test(
    view=view,
    app_ids=["website"],
)
data

### Applying the view to Signals

The following block pushes the view definition to the Signals API and makes it available for processing.

In [ ]:
applied = sp_signals.apply([view])
print(f"{len(applied)} objects applied")